In [ ]:
import pygame, sys, random
from pygame.locals import *

WINDOWWIDTH = 400 # chiều rộng window
WINDOWHEIGHT = 600 # chiều cao window

# load ảnh background
BACKGROUND = pygame.image.load('/Users/nguyenthihuyensam/GitHub/PYTHON/FLAPPY BIRD/img/background.png') 

BIRDWIDTH = 50 # chiều rộng bird
BIRDHEIGHT = 50 # chiều dài bird
G = 0.5 # gia tốc trọng trường (nếu rơi tự do thì G = 0.9)
SPEEDFLY = -8 # vận tốc bay lên của chim luôn âm
# load ảnh bird
BIRDIMG = pygame.image.load('/Users/nguyenthihuyensam/GitHub/PYTHON/FLAPPY BIRD/img/bird3.png') 

COLUMNWIDTH = 60 #chiều rộng cột
COLUMNHEIGHT = 500 # chiều cao cột
BLANK = 160 # khoảng trống giữa cột trên và cột dưới
DISTANCE = 200 # khoảng cách 2 cột
COLUMNSPEED = 2 # tốc độ di chuyển của cột
# load ảnh column
COLUMNIMG = pygame.image.load('/Users/nguyenthihuyensam/GitHub/PYTHON/FLAPPY BIRD/img/column.png')

pygame.init() # Khởi tạo pygame
FPS = 60 #  frames-per-second : số khung hình hiển thị mỗi giây
fpsClock = pygame.time.Clock() # set time 

DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT)) # hiển thị screen
pygame.display.set_caption('Flappy Bird') # đặt caption

# Tạo lớp bird
class Bird():
    # Hàm dựng (hàm khởi tạo)
    def __init__(self):
        self.width = BIRDWIDTH # set chiều rộng của bird
        self.height = BIRDHEIGHT # set chiều cao của bird
        self.x = (WINDOWWIDTH - self.width)/2 # toạ độ x của bird
        self.y = (WINDOWHEIGHT- self.height)/2 # toạ độ y của bird
        self.speed = 0 # set vận tốc ban đầu của bird = 0
        self.suface = BIRDIMG # set 

    # Hàm vẽ bird
    def draw(self):
        # hàm surface.blit() dùng để vẽ một vận trên một background
        DISPLAYSURF.blit(self.suface, (int(self.x), int(self.y)))
    
    # Tạo chuyển động bay lên, rơi xuống
    def update(self, mouseClick):
        # Tính toạ độ y dựa công thức của chuyển động rơi tự do
		# y = y0 + v0 * t + 0.5 * g * t^2
		# v = v0 + gt
		# ở đây chọn t = 1 
		# Vì mỗi vị trí toạ độ của chim ứng với một khung hình và 
		# mỗi khung hình cách nhau một khoảng thời gian nhất định 
        self.y += self.speed + 0.5 * G 
        self.speed += G                              
        # con chim bay lên sau mỗi lần click 
        if mouseClick == True:
            self.speed = SPEEDFLY # tức là khi click chuột v = - 8 

#Tạo lớp Column
class Columns():
    # Hàm khởi 
    def __init__(self):
        self.width = COLUMNWIDTH # Set chiều rộng cột
        self.height = COLUMNHEIGHT # Set chiều cao cột
        self.blank = BLANK # Set khoảng trống giữa cột trên và cột dưới
        self.distance = DISTANCE # Set khoảng cách giữa 2 cột 
        self.speed = COLUMNSPEED # Set tốc độ di chuyển của cột
        self.surface = COLUMNIMG # Set ảnh cái cột
        # Tạo 1 list để lưu trữ các cột và khoảng trống
        # [x, y] với x là hoành độ của cột, y là tung độ của khoảng trống
        self.ls = []
        for i in range(3): # list gồm 3 cột
            # các cột cách đều nhau một khoảng DISTANCE
            x = WINDOWWIDTH + i*self.distance
            # toạ độ y được tạo để blank ở các độ cao khác nhau
			# hàm random.range() tạo ngẫu nhiên 1 số trong khoảng
			# từ 60 đến WINDOWHEIGHT - self.blank - 60 và cách nhau 20
            y = random.randrange(60, WINDOWHEIGHT - self.blank - 60, 20)
            self.ls.append([x, y])
      # Vẽ cột   
    def draw(self):
        for i in range(3):
            # Vẽ cột trên background
            DISPLAYSURF.blit(self.surface, (self.ls[i][0], self.ls[i][1] - self.height))
            # set blank nên trên backgrou
            DISPLAYSURF.blit(self.surface, (self.ls[i][0], self.ls[i][1] + self.blank))
    
    # Tạo chuyển động cho 
    def update(self):
        # mảng chỉ có 3 phần tử nên khi xoá 1 cột thì phải thêm vào 1 cột
        for i in range(3):
            self.ls[i][0] -= self.speed
        # kiểm tra xem cột đầu tiên có đi ra ngoài màn hình không
        if self.ls[0][0] < -self.width:
            # nếu có thì xoá
            # rồi tạo ra 1 cột mới và thêm vào list
            self.ls.pop(0)
            x = self.ls[1][0] + self.distance
            y = random.randrange(60, WINDOWHEIGHT - self.blank - 60, 10)
            self.ls.append([x, y])
# Xử lý va chạm
# Va chạm giữa chim và cột thực chất là va chạm giữa 2 hình chữ nhật. 
# Hình chữ nhật thứ nhất là chim, hình chữ nhật thứ 2 là 1 phần cột trên hoặc dưới
# Đối truyền vào là 2 list gồm 4 thông số của hình chữ nhật (hoành độ, tung độ, chiều rộng, chiều cao) 
def rectCollision(rect1, rect2):
    if rect1[0] <= rect2[0]+rect2[2] and rect2[0] <= rect1[0]+rect1[2] and rect1[1] <= rect2[1]+rect2[3] and rect2[1] <= rect1[1]+rect1[3]:
        return True
    return False

# Kiểm tra game 
def isGameOver(bird, columns):
    for i in range(3):
        rectBird = [bird.x, bird.y, bird.width, bird.height]
        rectColumn1 = [columns.ls[i][0], columns.ls[i][1] - columns.height, columns.width, columns.height]
        rectColumn2 = [columns.ls[i][0], columns.ls[i][1] + columns.blank, columns.width, columns.height]
        if rectCollision(rectBird, rectColumn1) == True or rectCollision(rectBird, rectColumn2) == True:
            return True
    if bird.y + bird.height < 0 or bird.y + bird.height > WINDOWHEIGHT:
        return True
    return False

# Tạo class score 
class Score():
    def __init__(self):
        self.score = 0 # điểm ban đầu = 0
        self.addScore = True
    
    def draw(self):
        font = pygame.font.SysFont('consolas', 40) # set font chữ và cỡ chữ
        scoreSuface = font.render(str(self.score), True, (0, 0, 0)) # set chữ ở giữa màn hình
        textSize = scoreSuface.get_size() # set cỡ chữ
        # Chèn điểm trên background
        DISPLAYSURF.blit(scoreSuface, (int((WINDOWWIDTH - textSize[0])/2), 100))
    
    # Thay đổi điểm 
    def update(self, bird, columns):
        collision = False
        for i in range(3):
            rectColumn = [columns.ls[i][0] + columns.width, columns.ls[i][1], 1, columns.blank]
            rectBird = [bird.x, bird.y, bird.width, bird.height]
            if rectCollision(rectBird, rectColumn) == True:
                collision = True
                break
        if collision == True:
            if self.addScore == True:
                self.score += 1
            self.addScore = False
        else:
            self.addScore = True

def gameStart(bird):
    bird.__init__()

    font = pygame.font.SysFont('consolas', 60)
    headingSuface = font.render('FLAPPY BIRD', True, (255, 0, 0))
    headingSize = headingSuface.get_size()
    
    font = pygame.font.SysFont('consolas', 20)
    commentSuface = font.render('Click to start', True, (0, 0, 0))
    commentSize = commentSuface.get_size()
    
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == MOUSEBUTTONDOWN:
                return

        DISPLAYSURF.blit(BACKGROUND, (0, 0))
        bird.draw()
        DISPLAYSURF.blit(headingSuface, (int((WINDOWWIDTH - headingSize[0])/2), 100))
        DISPLAYSURF.blit(commentSuface, (int((WINDOWWIDTH - commentSize[0])/2), 500))

        pygame.display.update()
        fpsClock.tick(FPS)

def gamePlay(bird, columns, score):
    bird.__init__()
    bird.speed = SPEEDFLY
    columns.__init__()
    score.__init__()
    while True:
        mouseClick = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == MOUSEBUTTONDOWN:
                mouseClick = True
        
        DISPLAYSURF.blit(BACKGROUND, (0, 0))
        columns.draw()
        columns.update()
        bird.draw()
        bird.update(mouseClick)
        score.draw()
        score.update(bird, columns)

        if isGameOver(bird, columns) == True:
            return

        pygame.display.update()
        fpsClock.tick(FPS)

def gameOver(bird, columns, score):
    font = pygame.font.SysFont('consolas', 60)
    headingSuface = font.render('GAMEOVER', True, (255, 0, 0))
    headingSize = headingSuface.get_size()
    
    font = pygame.font.SysFont('consolas', 20)
    commentSuface = font.render('Press "space" to replay', True, (0, 0, 0))
    commentSize = commentSuface.get_size()

    font = pygame.font.SysFont('consolas', 30)
    scoreSuface = font.render('Score: ' + str(score.score), True, (0, 0, 0))
    scoreSize = scoreSuface.get_size()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYUP:
                if event.key == K_SPACE:
                    return
        
        DISPLAYSURF.blit(BACKGROUND, (0, 0))
        columns.draw()
        bird.draw()
        DISPLAYSURF.blit(headingSuface, (int((WINDOWWIDTH - headingSize[0])/2), 100))
        DISPLAYSURF.blit(commentSuface, (int((WINDOWWIDTH - commentSize[0])/2), 500))
        DISPLAYSURF.blit(scoreSuface, (int((WINDOWWIDTH - scoreSize[0])/2), 160))

        pygame.display.update()
        fpsClock.tick(FPS)

def main():
    bird = Bird()
    columns = Columns()
    score = Score()
    while True:
        gameStart(bird)
        gamePlay(bird, columns, score)
        gameOver(bird, columns, score)

if __name__ == '__main__':
    main()